In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from models.addmul import HandleAddMul

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [3]:
network_cache_dir = "networks/cache-networks/"
network_name = "lyr256-split0.8-lr0.01-add-mul.data"

checkpoint = True
test_flag = 1

input_dims = [42]
output_dims = [20]
batchsize = 128
num_epochs = 1

handler = HandleAddMul(input_dims, output_dims, dir=network_cache_dir + network_name, checkpoint=checkpoint, lr=0.001)

... FNN Network training on cuda:0 ...
Accessing : networks/cache-networks/lyr256-split0.8-lr0.01-add-mul.data
networks/cache-networks/lyr256-split0.8-lr0.01-add-mul.data


## generate mask on addition

In [4]:
logits = []
for layer in handler.network.layers[0]:
    if isinstance(layer, torch.nn.Linear):
        logits.append(torch.full_like(layer.weight.data, 0.9, requires_grad=True))

logits_generator = iter(logits)
        
for name, param in handler.network.named_parameters():
    param.requires_grad_(requires_grad=False)
    
train_split = 0.8
test_split = 1 - train_split

data_fp = "generate_datasets/tmp/digit-data/simple_add.npy"
data = np.load(data_fp, allow_pickle=True)

data_len = len(data)
train_split_idx = int(data_len * train_split)
train_data = data[:train_split_idx]
test_data = data[train_split_idx:]

train_loader = torch.utils.data.DataLoader(dataset=torch.tensor(train_data), batch_size=batchsize, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=torch.Tensor(test_data), batch_size=batchsize, shuffle=True)

def cycle(iterable):
    while True:
        for x in iterable:
            yield x

iterator_train = iter(cycle(train_loader))
iterator_test = iter(cycle(test_loader))

criterion = torch.nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(logits, lr=0.01)

NUM_EPOCHS = 20000  # NB: check for number of training epochs in paper
tau = 1  # temperature parameter, NB: check for value in paper
alpha = 0.0001/128  # regularisation parameter, NB: check for value in paper

In [1]:
import copy
loss_hist = []
avg = []
NUM_EPOCHS = 20000

handler = HandleAddMul(input_dims, output_dims, dir=network_cache_dir + network_name, checkpoint=checkpoint, lr=0.001)

for e in range(NUM_EPOCHS):
    print(f'Starting epoch {e}...')
    
    '''Reload weights'''
    handler.network.load_save()
    handler.refreeze_weights()
    '''Sampling and generating masks.'''

    U1 = torch.rand(1, requires_grad=False).to(handler.network.device)
    U2 = torch.rand(1, requires_grad=False).to(handler.network.device)
    
    samples = []

    for layer in logits:
#         layer.requires_grad_(requires_grad=True)
        samples.append(torch.sigmoid((layer - torch.log(torch.log(U1) / torch.log(U2))) / tau))

    binaries_stop = []

    for layer in samples:
        with torch.no_grad():
            binaries_stop.append((layer > 0.5).float() - layer)

    binaries = []
    iterator_samples = iter(samples)

    for layer in binaries_stop:
        binaries.append(layer + next(iterator_samples))

    iterator_binaries = iter(binaries)

    for layer in handler.network.layers[0]:
        if isinstance(layer, torch.nn.Linear):
            layer.weight.data * next(iterator_binaries)

    '''Inference with masked network and backpropagation.'''
    optimiser.zero_grad()

    batch = next(iterator_train)
#     with torch.no_grad():
    # Load in batch data (not binaries for one-hot input)
    inp = torch.stack([torch.stack([b[0], b[1]]) for b in batch])
    otp = torch.stack([b[2] for b in batch])
    ops = torch.stack([b[3] for b in batch])
    # Convert batch data toone-hot representation
    inp, otp_ = handler.set_batched_digits(inp, otp, ops)

    inp_ = inp.to(handler.network.device)
    otp_ = otp_.to(handler.network.device)

    otp_pred = handler.network(inp_)
    #otp_pred.requires_grad = True

        
    all_logits = alpha*torch.cat([layer.view(-1) for layer in logits]).to(handler.network.device)
    
    loss = criterion(otp_pred, otp_).to(handler.network.device) + torch.sum(all_logits).detach()
    #loss.requires_grad = True
    
    loss.backward()
    optimiser.step()

    loss_hist.append(loss.item())
    avg.append(np.mean(loss_hist[-100:]))
    if e % 100 == 0:
        plt.cla()
        plt.clf()
        #plt.plot(loss_hist)
        plt.plot(avg)
        plt.savefig('liveplot.png')
        plt.cla()
        plt.clf()
        plt.close()
    if e % 2000 == 0:
        torch.save(logits, 'masks/trained_logits_add_mask.pt')

NameError: name 'HandleAddMul' is not defined

## generate mask on multiplication

In [ ]:
train_split = 0.8
test_split = 1 - train_split

data_fp = "generate_datasets/tmp/digit-data/simple_mul.npy"
data = np.load(data_fp, allow_pickle=True)

data_len = len(data)
train_split_idx = int(data_len * train_split)
train_data = data[:train_split_idx]
test_data = data[train_split_idx:]

train_loader = torch.utils.data.DataLoader(dataset=torch.tensor(train_data), batch_size=batchsize, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=torch.Tensor(test_data), batch_size=batchsize, shuffle=True)

def cycle(iterable):
    while True:
        for x in iterable:
            yield x

iterator_train = iter(cycle(train_loader))
iterator_test = iter(cycle(test_loader))

criterion = torch.nn.CrossEntropyLoss()

optimiser = torch.optim.Adam(logits, lr=0.01)

NUM_EPOCHS = 20000  # NB: check for number of training epochs in paper
tau = 1  # temperature parameter, NB: check for value in paper
alpha = 0.0001/128  # regularisation parameter, NB: check for value in paper

In [ ]:
loss_hist = []
NUM_EPOCHS = 20000
for e in range(NUM_EPOCHS):
    print(f'Starting epoch {e}...')

    '''Sampling and generating masks.'''

    U1 = torch.rand(1, requires_grad=True).to(handler.network.device)
    U2 = torch.rand(1, requires_grad=True).to(handler.network.device)

    samples = []

    for layer in logits:
        layer.requires_grad_(requires_grad=True)

        #         if layer.grad is not None:
        #             layer.grad.detach_()
        #             layer.grad.zero_()

        samples.append(torch.sigmoid((layer - torch.log(torch.log(U1) / torch.log(U2))) / tau))

    binaries_stop = []

    for layer in samples:
        with torch.no_grad():
            binaries_stop.append((layer > 0.5).float() - layer)

    binaries = []
    iterator_samples = iter(samples)

    for layer in binaries_stop:
        binaries.append(layer + next(iterator_samples))

    iterator_binaries = iter(binaries)

    for layer in handler.network.layers[0]:
        if isinstance(layer, torch.nn.Linear):
            layer.weight.data * next(iterator_binaries)

    '''Inference with masked network and backpropagation.'''

    batch = next(iterator_train)

    with torch.no_grad():
        # Load in batch data (not binaries for one-hot input)
        inp = torch.stack([torch.stack([b[0], b[1]]) for b in batch])
        otp = torch.stack([b[2] for b in batch])
        ops = torch.stack([b[3] for b in batch])
        # Convert batch data toone-hot representation
        inp, otp_ = handler.set_batched_digits(inp, otp, ops)
        
        inp_ = inp.to(handler.network.device)
        otp_ = otp_.to(handler.network.device)
        
        otp_pred = handler.network(inp_)

        
    all_logits = alpha*torch.cat([layer.view(-1) for layer in logits]).to(handler.network.device)
    optimiser.zero_grad()
    
    loss = criterion(otp_pred, otp_).to(handler.network.device) + torch.sum(all_logits)
    #loss.requires_grad = True
    
    loss.backward()
    optimiser.step()

    loss_hist.append(loss.item())
    
    if e % 200 == 0:
        plt.cla()
        plt.clf()
        plt.plot(loss_hist)
        plt.savefig('liveplot.png')
        plt.cla()
        plt.clf()
        plt.close()
        torch.save(logits, 'masks/trained_logits_mul_mask.pt')